In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/others/new_credits.csv')
df_titles = pd.read_csv('../data/titles.csv')
filter_country = df['iso'].notna()
df_country = df[filter_country]

df_country.insert(10, "imdb", [0] * len(df_country))
df_country.insert(10, "tmdb", [0] * len(df_country))
for index, row in df_titles.iterrows():
    try:
        filter_id = df_country['id'] == row['id']
        df_title = df_country[filter_id]
        if len(df_country[filter_id]) == 0:
            continue
        df_country.loc[filter_id, 'imdb'] = row['imdb_score']
        df_country.loc[filter_id, 'tmdb'] = row['tmdb_score']
    except KeyError:
        continue
df_country = df_country[df_country['imdb'].notna() & df_country['tmdb'].notna()]
df_country.to_csv("../data/others/new_credits_score.csv")


/tmp/ipykernel_55940/2066397806.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '8.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_country.loc[filter_id, 'imdb'] = row['imdb_score']
/tmp/ipykernel_55940/2066397806.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '8.179' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_country.loc[filter_id, 'tmdb'] = row['tmdb_score']


In [3]:
person_ids = df_country['person_id'].unique().tolist()

country_mean_imdb_scores = dict()
country_mean_tmdb_scores = dict()
for person_id in person_ids:
    filter_person_id = df_country['person_id'] == person_id
    df_person = df_country[filter_person_id]
    mean_imdb = df_person.loc[:, 'imdb'].mean()
    mean_tmdb = df_person.loc[:, 'tmdb'].mean()
    
    iso = df_person.iloc[0]['iso']
    if iso not in country_mean_imdb_scores.keys():
        country_mean_imdb_scores[iso] = [mean_imdb]
        country_mean_tmdb_scores[iso] = [mean_tmdb]
    else:
        country_mean_imdb_scores[iso].append(mean_imdb)
        country_mean_tmdb_scores[iso].append(mean_tmdb)

df_country_mean_scores = pd.DataFrame(columns=['iso', 'mean_imdb', 'mean_tmdb'])
for iso in country_mean_imdb_scores.keys():
    mean_imdb = sum(country_mean_imdb_scores[iso]) / len(country_mean_imdb_scores[iso])
    mean_tmdb = sum(country_mean_tmdb_scores[iso]) / len(country_mean_tmdb_scores[iso])
    row = [iso, round(mean_imdb, 2), round(mean_tmdb, 2)]
    df_country_mean_scores.loc[len(df_country_mean_scores.index)] = row

df_country_mean_scores.to_csv('../data/country_scores.csv')